In [13]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression

Propensity Score Matching (PSM) — метод оцінки ефекту в нерандомізованих дослідженнях

- PSM використовується, коли групи не рандомізовані і потрібно скоригувати систематичні відмінності між ними.
- Основна ідея: обчислити ймовірність потрапити в “лікувальну” групу (propensity score) на основі характеристик учасників, а потім підбирати схожих учасників із контрольної групи.
- Outcome (результат) порівнюють після підбору, щоб оцінити ефект.
- Якщо поділ рандомний, як у класичному RCT, PSM не потрібен, бо групи вже збалансовані.
- PSM допомагає уникнути спотворень через відмінності у характеристиках, які можуть впливати на результат.

In [14]:
df = pd.read_csv('data/user_data_2000.csv')
df['Group'] = np.where(df['Group'] == 'Test', 1, 0)
df

,User_ID,Group,Retention_7d,Retention_30d,Avg_Session_Time,Region
0,101,1,0,1,7,EU
1,102,0,1,0,13,US
2,103,1,1,0,9,US
3,104,1,1,0,12,EU
4,105,1,1,1,11,EU
...,...,...,...,...,...,...
1995,2096,1,1,1,5,US
1996,2097,0,1,0,6,Asia
1997,2098,1,1,1,16,US
1998,2099,1,1,1,10,EU


In [15]:
X = pd.concat([
    df[['Avg_Session_Time']],                     
    pd.get_dummies(df['Region'], drop_first=True) 
], axis=1)
y = df['Retention_30d']

pd.concat([X, y], axis=1)

,Avg_Session_Time,EU,US,Retention_30d
0,7,True,False,1
1,13,False,True,0
2,9,False,True,0
3,12,True,False,0
4,11,True,False,1
...,...,...,...,...
1995,5,False,True,1
1996,6,False,False,0
1997,16,False,True,1
1998,10,True,False,1


In [16]:
logit_model = LogisticRegression()
logit_model.fit(X, y)
df['propensity_score'] = logit_model.predict_proba(X)[:, 1]
df

,User_ID,Group,Retention_7d,Retention_30d,Avg_Session_Time,Region,propensity_score
0,101,1,0,1,7,EU,0.503455
1,102,0,1,0,13,US,0.480842
2,103,1,1,0,9,US,0.493296
3,104,1,1,0,12,EU,0.487879
4,105,1,1,1,11,EU,0.490993
...,...,...,...,...,...,...,...
1995,2096,1,1,1,5,US,0.505758
1996,2097,0,1,0,6,Asia,0.511002
1997,2098,1,1,1,16,US,0.471516
1998,2099,1,1,1,10,EU,0.494108


In [17]:
test = df[df['Group'] == 1].copy()
control = df[df['Group'] == 0].copy()

nbrs = NearestNeighbors(n_neighbors=1)
nbrs.fit(control[['propensity_score']])
distances, indices = nbrs.kneighbors(test[['propensity_score']])

matched_control = control.iloc[indices.flatten()].copy()
matched_test = test.copy()

In [18]:
t7, p7 = ttest_ind(matched_test['Retention_7d'], matched_control['Retention_7d'], equal_var=False)
t30, p30 = ttest_ind(matched_test['Retention_30d'], matched_control['Retention_30d'], equal_var=False)

print(f"7d Retention: test_mean={matched_test['Retention_7d'].mean():.3f}, control_mean={matched_control['Retention_7d'].mean():.3f}, t={t7:.3f}, p={p7:.3f}")
print(f"30d Retention: test_mean={matched_test['Retention_30d'].mean():.3f}, control_mean={matched_control['Retention_30d'].mean():.3f}, t={t30:.3f}, p={p30:.3f}")

7d Retention: test_mean=0.506, control_mean=0.492, t=0.621, p=0.535
30d Retention: test_mean=0.465, control_mean=0.636, t=-7.873, p=0.000
